<a href="https://colab.research.google.com/github/nakamae11/-/blob/main/%E6%A5%BD%E5%A4%A9%E3%82%B5%E3%83%BC%E3%83%81%E6%9C%9F%E9%96%93%E6%8C%87%E5%AE%9A%E5%8F%AF%E8%83%BDVer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title 楽天商品検索API＋スプレッドシート出力（レビューAPI連携は申請後対応）

import requests
import gspread
from google.colab import auth
from google.auth import default
from IPython.display import display
import ipywidgets as widgets
import datetime

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# スプレッドシート名
SPREADSHEET_NAME = 'rakuten_search_result'

# 楽天API設定
RAKUTEN_APP_ID = '1029815826014097771'

# パラメータ入力ウィジェット
keyword_widget = widgets.Text(value='コーヒー', description='キーワード:')
sort_widget = widgets.Dropdown(options=[
    ('標準', 'standard'),
    ('レビュー件数が多い順', 'reviewCount'),
    ('販売数が多い順', 'sales'),
    ('価格が安い順', '+itemPrice'),
    ('価格が高い順', '-itemPrice'),
], description='ソート:')
hits_widget = widgets.IntText(value=30, description='取得件数:')
review_days_widget = widgets.IntText(value=30, description='期間(日):')

button = widgets.Button(description="実行")

display(keyword_widget, sort_widget, hits_widget, review_days_widget, button)

def main(_):
    keyword = keyword_widget.value
    sort = sort_widget.value
    hits = hits_widget.value
    review_days = review_days_widget.value  # まだ未対応（レビューAPI申請後に実装）

    # 楽天商品検索API
    url = 'https://app.rakuten.co.jp/services/api/IchibaItem/Search/20170706'
    params = {
        'applicationId': RAKUTEN_APP_ID,
        'keyword': keyword,
        'sort': sort,
        'hits': hits,
        'format': 'json'
    }
    res = requests.get(url, params=params).json()

    # スプレッドシート準備
    try:
        sheet = gc.open(SPREADSHEET_NAME).sheet1
    except gspread.SpreadsheetNotFound:
        sheet = gc.create(SPREADSHEET_NAME).sheet1

    sheet.clear()
    sheet.append_row(["商品名", "価格", "レビュー数", "商品URL", "ショップ名", "ショップURL"])

    for item in res.get("Items", []):
        data = item["Item"]
        row = [
            data["itemName"],
            data["itemPrice"],
            data["reviewCount"],
            data["itemUrl"],
            data["shopName"],
            data["shopUrl"]
        ]
        sheet.append_row(row)

    print(f"{len(res.get('Items', []))} 件の商品をスプレッドシートに書き込みました。")
    print(f"レビューAPI連携は申請完了後に対応します。")

button.on_click(main)


Text(value='コーヒー', description='キーワード:')

Dropdown(description='ソート:', options=(('標準', 'standard'), ('レビュー件数が多い順', 'reviewCount'), ('販売数が多い順', 'sales'),…

IntText(value=30, description='取得件数:')

IntText(value=30, description='期間(日):')

Button(description='実行', style=ButtonStyle())

30 件の商品をスプレッドシートに書き込みました。
レビューAPI連携は申請完了後に対応します。
